In [1]:
# Import libraries
import pandas as pd
from statsbombpy import sb

In [ ]:
matchesDF = sb.matches(competition_id=9, season_id=281)
team = 'Bayer Leverkusen'

# Get the event data from all matches and store in a dataframe
events = []
for i in matchesDF['match_id']:
    m_events = sb.events(i)
    m_events['match_id'] = i # Column for the matchid
    events.append(m_events)

# Join the dataframes
matchEvents = pd.concat(events, axis=0, ignore_index=True)

# Split columns
matchEvents[['location_x', 'location_y']] = matchEvents['location'].apply(pd.Series)
matchEvents[['pass_end_location_x', 'pass_end_location_y']] = matchEvents['pass_end_location'].apply(pd.Series)
matchEvents['shot_end_location'] = matchEvents['shot_end_location'].astype(str)
matchEvents[['shot_end_location_x', 'shot_end_location_y', 'shot_end_location_z']] = matchEvents['shot_end_location'].str.strip('[]').str.split(', ', expand=True)

Get evetns for Florian Wirtz

In [8]:
player = 'Florian Wirtz'
df = matchEvents[matchEvents['player'] == player]

# Get goal contribution events
florianDF = df[(df['shot_outcome'] == 'Goal') | (df['pass_goal_assist'] == True)]
florianEvents = florianDF[['id','match_id','minute','player','player_id',
        'shot_outcome', 'pass_goal_assist','shot_statsbomb_xg','location_x', 'location_y','pass_end_location_x','pass_end_location_y']]
#florianEvents['player_id'] = florianEvents['player_id'].astype(int)

florianEvents['goal_outcome'] = florianEvents.apply(
    lambda row: row['shot_outcome'] if pd.notna(row['shot_outcome']) else ('Assist' if row['pass_goal_assist'] else None),
    axis=1
)

# Export
#florianEvents.to_csv(r'./Player Goal Contribution.csv', index=False)

florianEvents.head()

C:\Users\Razak\AppData\Local\Temp\ipykernel_22816\610396780.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  florianEvents['goal_outcome'] = florianEvents.apply(


,id,match_id,minute,player,player_id,shot_outcome,pass_goal_assist,shot_statsbomb_xg,location_x,location_y,pass_end_location_x,pass_end_location_y,goal_outcome
4112,5085715b-09f6-403c-8b67-a546bcfed0ec,3895302,67,Florian Wirtz,40724.0,Goal,NaN,0.042012,93.5,40.1,NaN,NaN,Goal
4118,4a223f2d-3c3f-4655-b561-9c43a03b0f42,3895302,82,Florian Wirtz,40724.0,Goal,NaN,0.438327,103.6,35.0,NaN,NaN,Goal
4119,dcc0c591-7ebd-4e42-8206-04bfefa94f33,3895302,89,Florian Wirtz,40724.0,Goal,NaN,0.289998,106.4,48.8,NaN,NaN,Goal
7906,f5dea30a-5df6-4639-94dd-8319d9c8b256,3895292,52,Florian Wirtz,40724.0,Goal,NaN,0.783500,108.0,40.0,NaN,NaN,Goal
23637,9666f5e8-fa67-4ca1-a2a7-befeac0329e9,3895220,51,Florian Wirtz,40724.0,NaN,True,NaN,98.6,46.3,109.1,52.0,Assist


In [ ]:
# Get all player_id, player_name, jersey_number and country for all fixtures
players_list = []

for i in matchesDF['match_id']:
    lineup = sb.lineups(match_id = i)[team]
    fullLineup = lineup[['player_id','player_name','jersey_number','country']]
    players_list.append(fullLineup)

players_df = pd.concat(players_list, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)

# Export players info
# players_df.to_csv(r'./PlayersList.csv', index=False)

In [10]:
players_df

,player_id,player_name,jersey_number,country
0,3500,Granit Xhaka,34,Switzerland
1,7044,Patrik Schick,14,Czech Republic
2,8221,Jonathan Tah,4,Germany
3,8667,Lukáš Hrádecký,1,Finland
4,8804,Jonas Hofmann,7,Germany
5,9195,Robert Andrich,8,Germany
6,10336,Alejandro Grimaldo García,20,Spain
7,27133,Odilon Kossonou,6,Côte d'Ivoire
8,28268,Exequiel Alejandro Palacios,25,Argentina
9,30606,Edmond Fayçal Tapsoba,12,Burkina Faso
